In [6]:
import pandas as pd



In [73]:
path = "/Users/liangtelkamp/Documents/GitHub/un-sdd/processing/tests/test2.xlsx"
df = pd.read_excel(path)
df.to_json()


'{"Unnamed: 0":{"0":"Table 3: Number of acreage under irrigation","1":null,"2":null,"3":null,"4":null,"5":"What is the average size of land you own that is currently under irrigation?","6":null,"7":null,"8":null,"9":null,"10":null},"Unnamed: 1":{"0":null,"1":null,"2":null,"3":null,"4":null,"5":"0 - 2 acres","6":"2 - 5 acres","7":"5 - 10 acres","8":"More than 10 acres","9":null,"10":"Total"},"Unnamed: 2":{"0":null,"1":"OVERALL","2":null,"3":null,"4":"N","5":22,"6":6,"7":1,"8":0,"9":760,"10":789},"Unnamed: 3":{"0":null,"1":null,"2":null,"3":null,"4":"%","5":"2.8%","6":".8%","7":".1%","8":"0.0%","9":"96.3%","10":"100.0%"},"Unnamed: 4":{"0":null,"1":"Sub county","2":"Chepalungu","3":"Male","4":"N","5":4,"6":2,"7":0,"8":0,"9":176,"10":182},"Unnamed: 5":{"0":null,"1":null,"2":null,"3":null,"4":"%","5":"2.2%","6":"1.1%","7":"0.0%","8":"0.0%","9":"96.7%","10":"100.0%"},"Unnamed: 6":{"0":null,"1":null,"2":null,"3":"Female","4":"N","5":10,"6":2,"7":0,"8":0,"9":251,"10":263},"Unnamed: 7":{"0":nul

In [72]:
import pandas as pd

def concatenate_multirow_headers_auto(df, sep=' | ', max_header_rows=10, threshold=0.5, special_symbols=None):
    if special_symbols is None:
        special_symbols = ['N', '%']  # add more symbols if needed
    
    n_rows = df.shape[0]
    
    # Step 1: Find the first row with >= threshold non-NaN values
    for i in range(min(max_header_rows, n_rows)):
        non_nan_fraction = df.iloc[i].notna().mean()
        if non_nan_fraction >= threshold:
            first_header_row = i
            break
    else:
        first_header_row = 0  # fallback if none meet threshold

    # Step 2: Include subsequent rows that are mostly NaN or contain special symbols
    last_header_row = first_header_row
    for i in range(first_header_row + 1, n_rows):
        row = df.iloc[i].astype(str)
        contains_symbol = any(symbol in row.values for symbol in special_symbols)
        mostly_empty = row.notna().mean() < threshold
        if mostly_empty or contains_symbol:
            last_header_row = i
        else:
            break

    # Step 3: Concatenate header rows
    headers = df.iloc[:last_header_row + 1].fillna(method='ffill', axis=1)
    concatenated_headers = headers.apply(lambda col: sep.join([str(i) for i in col if str(i) != 'nan']), axis=0)
    
    # Step 4: Set headers and drop header rows from data
    df.columns = concatenated_headers
    df = df.iloc[last_header_row + 1:].reset_index(drop=True)
    
    return df

# Usage example
df = pd.read_excel(path, header=None)
df_clean = concatenate_multirow_headers_auto(df)
df_clean


/var/folders/lm/8lf06d816nv8ym2rv97d1v580000gn/T/ipykernel_80823/2508873200.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  headers = df.iloc[:last_header_row + 1].fillna(method='ffill', axis=1)


,Table 3: Number of acreage under irrigation,Table 3: Number of acreage under irrigation,Table 3: Number of acreage under irrigation | OVERALL | N,Table 3: Number of acreage under irrigation | OVERALL | %,Table 3: Number of acreage under irrigation | Sub county | Chepalungu | Male | N,Table 3: Number of acreage under irrigation | Sub county | Chepalungu | Male | %,Table 3: Number of acreage under irrigation | Sub county | Chepalungu | Female | N,Table 3: Number of acreage under irrigation | Sub county | Chepalungu | Female | %,Table 3: Number of acreage under irrigation | Sub county | Bomet Central | Male | N,Table 3: Number of acreage under irrigation | Sub county | Bomet Central | Male | %,Table 3: Number of acreage under irrigation | Sub county | Bomet Central | Female | N,Table 3: Number of acreage under irrigation | Sub county | Bomet Central | Female | %
0,What is the average size of land you own that ...,0 - 2 acres,22,2.8%,4,2.2%,10,3.8%,3,1.7%,5,2.9%
1,NaN,2 - 5 acres,6,.8%,2,1.1%,2,.8%,0,0.0%,2,1.2%
2,NaN,5 - 10 acres,1,.1%,0,0.0%,0,0.0%,0,0.0%,1,.6%
3,NaN,More than 10 acres,0,0.0%,0,0.0%,0,0.0%,0,0.0%,0,0.0%
4,NaN,NaN,760,96.3%,176,96.7%,251,95.4%,170,98.3%,163,95.3%
5,NaN,Total,789,100.0%,182,100.0%,263,100.0%,173,100.0%,171,100.0%


In [77]:
import pandas as pd
import json

# Load your JSON string
data_json = '{"Unnamed: 0":{"0":"Table 3: Number of acreage under irrigation","1":null,"2":null,"3":null,"4":null,"5":"What is the average size of land you own that is currently under irrigation?","6":null,"7":null,"8":null,"9":null,"10":null},"Unnamed: 1":{"0":null,"1":null,"2":null,"3":null,"4":null,"5":"0 - 2 acres","6":"2 - 5 acres","7":"5 - 10 acres","8":"More than 10 acres","9":null,"10":"Total"},"Unnamed: 2":{"0":null,"1":"OVERALL","2":null,"3":null,"4":"N","5":22,"6":6,"7":1,"8":0,"9":760,"10":789},"Unnamed: 3":{"0":null,"1":null,"2":null,"3":null,"4":"%","5":"2.8%","6":".8%","7":".1%","8":"0.0%","9":"96.3%","10":"100.0%"},"Unnamed: 4":{"0":null,"1":"Sub county","2":"Chepalungu","3":"Male","4":"N","5":4,"6":2,"7":0,"8":0,"9":176,"10":182},"Unnamed: 5":{"0":null,"1":null,"2":null,"3":null,"4":"%","5":"2.2%","6":"1.1%","7":"0.0%","8":"0.0%","9":"96.7%","10":"100.0%"},"Unnamed: 6":{"0":null,"1":null,"2":null,"3":"Female","4":"N","5":10,"6":2,"7":0,"8":0,"9":251,"10":263},"Unnamed: 7":{"0":null,"1":null,"2":null,"3":null,"4":"%","5":"3.8%","6":".8%","7":"0.0%","8":"0.0%","9":"95.4%","10":"100.0%"},"Unnamed: 8":{"0":null,"1":null,"2":"Bomet Central","3":"Male","4":"N","5":3,"6":0,"7":0,"8":0,"9":170,"10":173},"Unnamed: 9":{"0":null,"1":null,"2":null,"3":null,"4":"%","5":"1.7%","6":"0.0%","7":"0.0%","8":"0.0%","9":"98.3%","10":"100.0%"},"Unnamed: 10":{"0":null,"1":null,"2":null,"3":"Female","4":"N","5":5,"6":2,"7":1,"8":0,"9":163,"10":171},"Unnamed: 11":{"0":null,"1":null,"2":null,"3":null,"4":"%","5":"2.9%","6":"1.2%","7":".6%","8":"0.0%","9":"95.3%","10":"100.0%"}}'
df = pd.DataFrame(json.loads(data_json))

# Drop irrelevant rows (table title)
df = df.drop(["0","1","2","3","4"])


# Flatten multi-level headers manually
df.columns = [
    'Question',
    '0-2_acres',
    'Overall_N',
    'Overall_pct',
    'Chepalungu_Male_N',
    'Chepalungu_Male_pct',
    'Chepalungu_Female_N',
    'Chepalungu_Female_pct',
    'BometCentral_Male_N',
    'BometCentral_Male_pct',
    'BometCentral_Female_N',
    'BometCentral_Female_pct'
]

# Reset index
df = df.reset_index(drop=True)

# Optional: convert numeric columns to numbers
numeric_cols = df.columns[2:]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

df



,Question,0-2_acres,Overall_N,Overall_pct,Chepalungu_Male_N,Chepalungu_Male_pct,Chepalungu_Female_N,Chepalungu_Female_pct,BometCentral_Male_N,BometCentral_Male_pct,BometCentral_Female_N,BometCentral_Female_pct
0,What is the average size of land you own that ...,0 - 2 acres,22,NaN,4,NaN,10,NaN,3,NaN,5,NaN
1,None,2 - 5 acres,6,NaN,2,NaN,2,NaN,0,NaN,2,NaN
2,None,5 - 10 acres,1,NaN,0,NaN,0,NaN,0,NaN,1,NaN
3,None,More than 10 acres,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN
4,None,None,760,NaN,176,NaN,251,NaN,170,NaN,163,NaN
5,None,Total,789,NaN,182,NaN,263,NaN,173,NaN,171,NaN
